# Deep MTJ - automatic tracking of the muscel-tendon junction

This notebook shows how to use the MTJ tracking tool. Google Colab offers free online computation resources. You can track your own videos by uploading your video files (see menu on the left) and running the model online. It is also possible to run the tool on your local enviroment by installing the `setup.py` file (see GitHub repository; https://github.com/luuleitner/deepMTJ).

The MTJ tool can be directly installed from the GitHub repository using PIP.

In [30]:
#!pip install git+https://github.com/luuleitner/deepMTJ

For tracking the MTJ positions from multiple videos we can simple use the `track_videos` function.

In [31]:
from mtj_tracking.predict.predict import track_videos # main tracking function
from mtj_tracking.data.loader import Frame, loadVideo, adjustFrame # helper for DeepMTJ
from matplotlib import pyplot as plt # plotting function

We start by downloading a sample video from the Esaote instrument.

In [32]:
#urlretrieve('', 'esaote_video.avi')

First we need to specify the region of interest within the video. For this we select the x/y-position and the width and height of the frame that we want to crop from video.

In [33]:
frame = Frame((185, 128, 580, 290)) # x, y, width, height

Let us check that we have selected the correct sub-frame by reading the video.

In [34]:
video_id, video = loadVideo('esaote_video.avi') # returns the video id and a list of frames
adjusted_video_frame = adjustFrame(video[0], frame, (128, 256)) # crops to specified frame

plt.subplot(121)
plt.imshow(video[0]) # plot the first frame
plt.title('Full Frame')
plt.subplot(122)
plt.imshow(adjusted_video_frame, cmap='gray') # plot the adjusted frame
plt.title('Cropped Frame')
plt.show()

OpenCV: Couldn't read video stream from file "esaote_video.avi"
[ERROR:0] global ../modules/videoio/src/cap.cpp (162) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.2) ../modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): esaote_video.avi in function 'icvExtractPattern'




IndexError: list index out of range

Alternatively we can also use one of the predefined frames. Note that all images are processed at 256x128 pixels. The aspect ratio of the specified frame should be the same (2:1).
```
Frame.ESAOTE = (185, 128, 580, 290)
Frame.TELEMED = (88, 120, 446, 223)
Frame.AIXPLORER = (200, 261, 1000, 500)
```

Now we can track the MTJ in our video.

In [ ]:
result_df = track_videos(['esaote_video.avi'], Frame.ESAOTE)
result_df

Let us check the result of the model by plotting the predicted MTJ position.

In [ ]:
plt.imshow(adjusted_video_frame, cmap='gray') # plot the adjusted frame
plt.scatter(result_df.x[0], result_df.y[0], color='red')
plt.show()

The same check can be performed for the full video sequence.

In [ ]:
for i in range(len(video)):
    last_video_frame = adjustFrame(video[i], frame, (128, 256)) # crops to specified frame
    plt.imshow(last_video_frame, cmap='gray') # plot the adjusted frame
    plt.axis('off')
    plt.scatter(result_df.x[i], result_df.y[i], color='red')
    plt.show()

The result can be also converted to the pixel coordinates of the original video.

In [ ]:
result_oc_df = track_videos(['esaote_video.avi'], Frame.ESAOTE, frame_coordinates=False)
result_oc_df

In [ ]:
plt.imshow(video[0]) # plot the original frame
plt.scatter(result_oc_df.x[0], result_oc_df.y[0], color='red')
plt.show()

The list contains the pixel coordinates for each frame of the specified videos.

You can upload your own videos and convert them using the computational resources from Colab. If you have to process more data you can activate a GPU environment. For this select Runtime --> Change runtime type --> Hardware acceleration --> GPU

To convert your own files, upload your videos and specify the list of video files in the `track_video` function.
The result can be afterwards saved as a CSV file and downloaded.

In [ ]:
result_df = track_videos(['<<your-video>>'], Frame.ESAOTE)
result_df.to_csv('deep_mtj.csv')